In [1]:
# import libraries
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, BooleanType
import datetime
from datetime import datetime


StatementMeta(, 2f4bf558-a9b2-477e-8d54-164669657835, 3, Finished, Available, Finished)

In [1]:
%%sql
-- create materialization_log table in utilities_lakehouse
CREATE TABLE IF NOT EXISTS materialization_log (
    lakehouse_name STRING,
    schema_name STRING,
    table_name STRING,
    job_run_timestamp TIMESTAMP,
    run_id INT,
    run_step STRING,
    run_timestamp TIMESTAMP,
    record_count INT,
    step_fail STRING,
    statement_text STRING
) USING DELTA

StatementMeta(, 9a55aa02-fbdf-4cc0-a566-d25e24c4e925, 2, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
%%sql
-- create materialization load table
CREATE TABLE materialization_load (
    lakehouse_name STRING,
    input_source_schema STRING,
    input_source STRING,
    output_target_schema STRING, 
    output_target STRING,
    load_option STRING,
    load_type STRING,
    is_active BOOLEAN,
    date_created TIMESTAMP,
    last_execution TIMESTAMP,
    last_successful_execution TIMESTAMP
    ) USING DELTA

StatementMeta(, 2f4bf558-a9b2-477e-8d54-164669657835, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
%%sql
-- create materialization control table
CREATE TABLE materialization_control (
    lakehouse_name STRING,
    etl_database STRING,
    etl_view STRING,
    status_code STRING,
    extract_start_timestamp TIMESTAMP,
    extract_end_timestamp TIMESTAMP,
    script_header STRING,
    last_script_generate_timestamp TIMESTAMP,
    control_start_timestamp TIMESTAMP,
    load_option STRING,
    is_active BOOLEAN
) USING DELTA

StatementMeta(, 2f4bf558-a9b2-477e-8d54-164669657835, 6, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [1]:
# crate bunn_meta table in utilities lakehouse
spark.sql("""
CREATE TABLE IF NOT EXISTS bunn_meta (
    system_name STRING,
    source_schema STRING,
    source_table STRING,
    source_key STRING,
    target_schema STRING,
    target_table STRING,
    target_key STRING
)
USING delta
""")



StatementMeta(, 97b2a328-a4f3-4619-801b-7c6281a4243a, 3, Finished, Available, Finished)

DataFrame[]

In [1]:
# insert sample data into bunn_meta
spark.sql("""
INSERT INTO bunn_meta (system_name, source_schema, source_table, source_key, target_schema, target_table, target_key)
VALUES
    ('sapecc', 'materialized_etl', 'acct_doc_hdr_etl', 'client', 'materialized_t', 'acct_doc_hdr', 'client'),
    ('sapecc', 'materialized_etl', 'acct_doc_hdr_etl', 'company_code', 'materialized_t', 'acct_doc_hdr', 'company_code'),
    ('sapecc', 'materialized_etl', 'acct_doc_hdr_etl', 'acct_doc_nbr', 'materialized_t', 'acct_doc_hdr', 'acct_doc_nbr')
""")

StatementMeta(, be72956b-c014-44a6-99bc-daa4d7679560, 3, Finished, Available, Finished)

DataFrame[]

In [2]:
%%sql
select * from bunn_meta

StatementMeta(, be72956b-c014-44a6-99bc-daa4d7679560, 4, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 7 fields>

In [ ]:
%%sql
select * 
from materialization_log limit 10

StatementMeta(, 5410cd38-95ec-4d67-9874-776ed8c883f5, 5, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 10 fields>

In [9]:
#rename a column in utilities tables
df= spark.read.format("delta").load('abfss://BUNN_Foundation_NONPROD@onelake.dfs.fabric.microsoft.com/utilities_lakehouse.Lakehouse/Tables/materialization_control')

StatementMeta(, d3aeeeb3-1cf7-4e6d-834a-0ac3b9e82986, 11, Finished, Available, Finished)

In [8]:

df=df.withColumnRenamed("mtlz_database", "lakehousename")
display(df)

StatementMeta(, d3aeeeb3-1cf7-4e6d-834a-0ac3b9e82986, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b3df4823-9094-47eb-8ee4-877d3801fc7b)

In [3]:
# insert entries into materialization_load

spark.sql(f"""
INSERT INTO materialization_load
VALUES (
    'silver_sapecc_lakehouse',
    'materialized_etl',
    'acct_doc_hdr_etl',
    'materialized_t',
    'acct_doc_hdr',
    'CLDUI',
    null,
    1,
    current_timestamp(),
    current_timestamp(),
    current_timestamp()
)
""")

StatementMeta(, be72956b-c014-44a6-99bc-daa4d7679560, 5, Finished, Available, Finished)

DataFrame[]

In [6]:
# update entries into materialization_load

spark.sql(f"""
update materialization_load
set is_active = 0
where 
    input_source_schema = 'materialized_etl'
    and input_source = 'orders_etl'
""")

StatementMeta(, be72956b-c014-44a6-99bc-daa4d7679560, 8, Finished, Available, Finished)

DataFrame[num_affected_rows: bigint]

In [1]:
%%sql
select * from materialization_load
--2025-03-28T02:21:09Z, 2025-03-28T02:21:54Z

StatementMeta(, 8cf6c213-b698-42a9-b67d-2f3ad74d28c2, 2, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 11 fields>

In [2]:
%%sql
select * from materialization_control

StatementMeta(, 8cf6c213-b698-42a9-b67d-2f3ad74d28c2, 3, Finished, Available, Finished)

<Spark SQL result set with 6 rows and 11 fields>

In [3]:
%%sql
select * from materialization_log
order by job_run_timestamp desc, run_id desc limit 20

StatementMeta(, 8cf6c213-b698-42a9-b67d-2f3ad74d28c2, 4, Finished, Available, Finished)

<Spark SQL result set with 20 rows and 10 fields>

In [2]:
#spark.sql(f" delete from materialization_log")

df=spark.sql(f" select * from materialization_log").collect()[0][0]
print(f"row_count : {df}")

StatementMeta(, 3d84450c-77f8-4be8-8c6b-55c604b79f7b, 4, Finished, Available, Finished)

row_count : silver_sapecc_lakehouse


In [ ]:
%%sql
select lakehouse_name, input_source_schema, input_source, load_option,is_active
from materialization_load
--where is_active = 1

StatementMeta(, , -1, SessionError, , SessionError)

InvalidHttpRequestToLivy: [TooManyRequestsForCapacity] This spark job can't be run because you have hit a spark compute or API rate limit. To run this spark job, cancel an active Spark job through the Monitoring hub, choose a larger capacity SKU, or try again later. HTTP status code: 430 {Learn more} HTTP status code: 430.

In [ ]:
available_mounts = mssparkutils.fs.mounts()
print(available_mounts)

In [ ]:
# mount path of the attached lakehouse
lakehouse_path = mssparkutils.fs.getMountPath('default')
print("Lakehouse Path:", lakehouse_path)

In [ ]:
# create security rule table
spark.sql("""
CREATE TABLE IF NOT EXISTS access_role (
    env_id VARCHAR(20),
    role VARCHAR(100),
    user_id VARCHAR(100),
   -- column_id VARCHAR(100),
    rule VARCHAR(2000),
    is_active BOOLEAN,
    create_dt TIMESTAMP,
    update_dt TIMESTAMP
)
USING delta
""")

StatementMeta(, 5410cd38-95ec-4d67-9874-776ed8c883f5, 8, Finished, Available, Finished)

In [ ]:
# insert data
spark.sql(""" insert into table security_rule 
values (
    'prod', 'data_engineer', 'tmathew@oakwoodsys',"'DIST_CHANNEL_ID = '01' AND (SALES_GROUP_ID = '710' OR MS_BOOK_PART_NUMBER in ('COCA-COLA','FLORIDA NATURAL','VITALITY','SUBWAY','BURGER KING','POPEYES','NPV','CHICK-FIL-A'))'", 1, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP
        )"""
) 

StatementMeta(, 5410cd38-95ec-4d67-9874-776ed8c883f5, 9, Finished, Available, Finished)

In [ ]:
spark.sql("select current_user() as user").show()  # Shows Spark's effective user
spark.sql("select session_user() as user").show()  # Sometimes different

StatementMeta(, 0ca1901b-d392-4c8d-9b65-a3a0b8f7a8aa, 3, Finished, Available, Finished)

+--------------------+
|                user|
+--------------------+
|trusted-service-user|
+--------------------+



AnalysisException: [UNRESOLVED_ROUTINE] Cannot resolve function `session_user` on search path [`system`.`builtin`, `system`.`session`, `spark_catalog`.`utilities_lakehouse`].; line 1 pos 7

In [ ]:
user_id = spark.sql("select current_user() as user").collect()[0]
print(user_id)

StatementMeta(, 4b933246-2bd4-439f-ac3d-d3339e6af34f, 7, Finished, Available, Finished)

Row(user='trusted-service-user')


In [ ]:
import os
#print(os.environ.get('USER'))        # Linux/Mac
print(os.environ.get('USERNAME'))    # Windows

StatementMeta(, 4b933246-2bd4-439f-ac3d-d3339e6af34f, 9, Finished, Available, Finished)

None
